In [1]:
import deepmatcher as dm
import pandas as pd
import numpy as np

In [25]:
def getTrainValidationTest(dataset_path):
    df = pd.read_json(dataset_path,lines=True,compression='gzip')
    train, validation,test = np.split(df.sample(frac=1), [int(.6*len(df)),int(.8*len(df))])
    new_col = {}
    for col in list(train):
        if col.endswith('_left'):
            tokens = col.split('_left',1)
            new_col[col]='left_'+tokens[0]
        elif col.endswith('_right'):
            tokens = col.split('_right',1)
            new_col[col] = 'right_'+tokens[0]
        else:
            new_col[col] = col
    train = train.rename(columns = new_col)
    validation = validation.rename(columns =new_col)
    test = test.rename(columns = new_col)
    return train,validation,test

In [40]:
train,validation,test = getTrainValidationTest('WDCProductData/computer/computers_train_medium.json.gz')

In [42]:
ignore_columns = ['pair_id','left_title','left_description','left_brand','left_specTableContent',
                  'right_title','right_description','right_brand','right_specTableContent']
columns_to_drop = [col for col in list(train) if col not in ignore_columns]
train_df = train.drop(columns =columns_to_drop)
validation_df = validation.drop(columns =columns_to_drop)
test_df = test.drop(columns = columns_to_drop)

In [44]:
train_df.to_csv('computer_train.csv',index=False)
validation_df.to_csv('computer_validation.csv',index=False)
test_df.to_csv('computer_test.csv',index=False)

## Train model

In [45]:
train,validation,test = dm.data.process('./',train='computer_train.csv',validation='computer_validation.csv',
                                        test='computer_test.csv',ignore_columns=['left_id','right_id','pair_id'])

AssertionError: 

In [34]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.run_train(train,validation,epochs=20,pos_neg_ratio=4,best_save_path='models/computer_hybrid.pth')

* Number of trainable parameters: 19954811
===>  TRAIN Epoch 1


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:58

In [35]:
train.get_raw_table()

,left_title,left_description,left_brand,left_price,left_specTableContent,left_keyValuePairs,left_category,left_cluster_id,left_identifiers,right_title,right_description,right_brand,right_price,right_specTableContent,right_keyValuePairs,right_category,right_cluster_id,right_identifiers,label
0,amd black edition athlon ii x4 860k 3 7 ghz pr...,do more of the things you want to in less time...,,,,,computers_and_accessories,39139,"[ { '/gtin8 ' : ' [ 43201503 ] ' } , { '/mpn '...",amd athlon x4 870k 3 9 ghz black low noise edi...,amd athlon x4 870k amd athlon x4 socket fm2 pc...,,,temp rature d op ration 0 72 4 c socket de pro...,"{ 'temp rature d op ration ' : ' 0 72 4 c ' , ...",computers_and_accessories,94726,[ { '/sku ' : ' [ ad870kxbjcsbx ] ' } ],0
1,transcend 8gb usb flash drive 2 0 memory stora...,8gb usb flash drive,,,,,computers_and_accessories,351256,[ { '/mpn ' : ' [ ts8gjf500 ] ' } ],corsair flash voyager slider x2 usb drive 256 ...,,,,,,computers_and_accessories,946280,"[ { '/gtin8 ' : ' [ 43202010 ] ' } , { '/mpn '...",0
2,"null , xta 3510 73 gb 10k sun hp fc al hdd who...",description sun 73gb 10k rpm 3 5 inchhot plug ...,sun,,specifications category sun harddrive sub cate...,"{ 'category ' : 'sun harddrive ' , 'sub catego...",computers_and_accessories,2318154,"[ { '/sku ' : ' [ xta351073gb10k ] ' } , { '/m...","null , 359461 001 hp 72 8 gb 10k fc al hdd new...",description storageworks 72 8gb 10k fibre chan...,hp enterprise,,specifications category storageworks hdd sub c...,"{ 'category ' : 'storageworks hdd ' , 'sub cat...",computers_and_accessories,205792,"[ { '/sku ' : ' [ 359461001 ] ' } , { '/mpn ' ...",0
3,"483513 b21 hp ddr pci e dual port hca , null n...",description hp ib 4x ddr pci e dual port host ...,hp enterprise,,specifications category proliant controller su...,"{ 'category ' : 'proliant controller ' , 'sub ...",computers_and_accessories,3700218,"[ { '/sku ' : ' [ 483513b21 ] ' } , { '/mpn ' ...","376638 b21 hp 1gb pc3200 reg ecc kit , null",description genuine hpe 1gb memory kit 2 x 512...,hp enterprise,cad 90 94 cad,specifications category proliant memory sub ca...,"{ 'category ' : 'proliant memory ' , 'sub cate...",computers_and_accessories,349647,[ { '/mpn ' : ' [ 376638b21 ] ' } ],0
4,asus prime b250m a micro atx lga1151 motherboa...,,asus,,,,computers_and_accessories,170796,[ { '/mpn ' : ' [ primeb250ma ] ' } ],asus prime b250m a micro atx lga1151 motherboa...,,asus,,,,computers_and_accessories,170796,[ { '/mpn ' : ' [ primeb250ma ] ' } ],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851,2 5 inch force ls v2 120gb sata ssd corsair so...,120gb corsair force ls v2 series 2 5 ssd sata ...,,,,,computers_and_accessories,754769,[ { '/mpn ' : ' [ cssdf120gblsb ] ' } ],samsung 840 series 120gb 2 5 solid state drive...,,samsung,,,,computers_and_accessories,118302,[ { '/mpn ' : ' [ mz7td120bw ] ' } ],0
4852,2 5 inch 7mm 800gb intel data center s3610 sol...,800gb intel dc s3610 2 5 ssd sata 3 0 6gb s 20...,,,,,computers_and_accessories,148547,[ { '/mpn ' : ' [ ssdsc2bx800g401 ] ' } ],intel dc s3610 2 5 800gb prijzen tweakers,,,,categorie solid state drives merk intel produc...,"{ 'categorie ' : 'solid state drives ' , 'merk...",computers_and_accessories,148547,"[ { '/mpn ' : ' [ ssdsc2bx800g401 , ssdsc2bx80...",1
4853,zebra card printer zxp7 cod z71 0m0c0000em00,zebra zxp7 card printer color dye sublimation ...,zebra,,,,computers_and_accessories,11544927,[ { '/sku ' : ' [ z710m0c0000em00 ] ' } ],zebra stampante card zxp3 r2 enclosure lock se...,zebra zxp3 stampante di card stampa a trasferi...,zebra,,,,computers_and_accessories,7314007,[ { '/sku ' : ' [ z32a0ac0200em00 ] ' } ],0
4854,gigabyte geforce gtx 1070 windforce 2x oc 8192...,gv n1070wf2oc 8gd core clock 1582mhz boost clo...,gigabyte,,,,computers_and_accessories,425661,"[ { '/mpn ' : ' [ gvn1070wf2oc8gd ] ' } , { '/...",gigabyte geforce gtx 1070 mini itx oc graphics...,the gigabyte custom designed cooling system